In [1]:
# set working directory to parent
import os
os.chdir("..")

In [2]:
# import packages
import pandas as pd
import boto3
import numpy as np

# tensorflow
import tensorflow as tf
import tensorflow_hub as hub

# user modules
from src.data import process_data 
from src.models.architecture import ModelBuilder
from src.models.compile import ModelCompiler
from src.models.train import ModelTrainer
#from src.models.model_saver import save_model

# check tf version
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")


Version:  2.0.0
Eager mode:  True
Hub version:  0.7.0
GPU is NOT AVAILABLE


In [3]:
# load train.csv  
# split into 1) training, 2) test/validation_data
train_data,test_data = process_data.train_split(process_data.retrieve_training(bucket = "quora-questions", file_name = "data/train.csv"))

Examples:
    Total: 1306122
    Positive: 80810 (6.19% of total)



In [4]:
# build transfer learnig model 
model_builder = ModelBuilder()
transfer_gnews = model_builder.transfer_simple(embedding_layer = "gnews", num_mid_nodes = 16, model_name = "mriv_model0")

In [5]:
# model summary
transfer_gnews.summary()

Model: "mriv_model0"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_text (InputLayer)      [(None,)]                 0         
_________________________________________________________________
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 353
Non-trainable params: 400,020
_________________________________________________________________


In [6]:
# compile model --> hyperparameters
model_compiler = ModelCompiler()
transfer_gnews = model_compiler.compile_model(transfer_gnews)


In [8]:
# train model 
model_trainer = ModelTrainer(model = transfer_gnews, experiment_name = "mriv_model0_exp0")
transfer_gnews_history = model_trainer.train_history(train_data = train_data, validation_data = test_data)


FileExistsError: [Errno 17] File exists: '/home/mriveralanas/projects/quora/data/train_log/mriv_model0_exp0/'

In [ ]:
# plot key metrics over training history 


In [ ]:
# save serialized model

# hd5 file includes :
# The model's architecture
# The model's weight values (which were learned during training)
# The model's training config (what you passed to compile), if any
# The optimizer and its state, if any (this enables you to restart training where you left)

model_trainer.save_model()

In [ ]:
# test saved model and plot history 
